In [5]:
import os
import io
import pprint

import pandas as pd

from bson.objectid import ObjectId
import pymongo

import climatools.lblnew.bestfit_params as bestfits
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.setup_overlap as setup_overlap
import climatools.lblnew.pipeline as lblnew_pipeline

import climatools.cliradlw.setup as setup_clirad
import climatools.cliradlw.pipeline as clirad_pipeline

In [2]:
client = pymongo.MongoClient('localhost:27017')

In [3]:
db = client['kdistnew']

bestfit = db['bestfit']
overlap = db['overlap']
clirad = db['clirad']

In [84]:
def bestfit_outputfiles():
    return ['kg_nonlin.dat',
            'kg_lin.dat',
            'output_wcoolrg.dat',
            'output_wfluxg.dat',
            'output_flux.dat',
            'output_fluxg.dat',
            'output_coolr.dat',
            'output_coolrg.dat',
            'abscom.dat',
            'dgdgs.dat']


def insert_all_bestfits():
    for molecule, band in setup_bestfit.implemented_moleculebands():
        param = bestfits.kdist_params(molecule=molecule, band=band)
        dir_fortran = lblnew_pipeline.get_dir_case(param, 
                                                   setup=setup_bestfit)
        
        c_output = {}
        for name in bestfit_outputfiles():
            with open(os.path.join(dir_fortran, name), 
                      mode='r', encoding='utf-8') as f:
                c = f.read()      
        c_output[name.split('.')[0]] = c
        
        case = {'input': param, 'output': c_output}    
        bestfit.insert_one(case)

In [ ]:
def insert_all_cliradlw():
    molecule = 'h2o'
    band = '3'
    
    

In [100]:
bestfit.delete_one({'_id': ObjectId('5b28a260af10727d0e6a907c')})

In [4]:
for r in bestfit.find({'input.band': '3b'},
                      projection={'input.molecule': 1,
                                  'input.band': 1,
                                  'input.atmpro': 1,
                                  'input.commitnumber': 1,
                                  'output.dgdgs': 1}):
    print(r['_id'])
    pprint.pprint(r['input'])
    print(r['output']['dgdgs'])

5b28b908af10727d0e6a9081
{'atmpro': 'trp', 'band': '3b', 'commitnumber': 'a06b618', 'molecule': 'h2o'}
       g                  dg                 dgs
       1      0.74500000E-02      0.76657311E-02
       2      0.38520000E-01      0.38841090E-01
       3      0.92910000E-01      0.93707735E-01
       4      0.24753000E+00      0.24927679E+00
       5      0.38811000E+00      0.39011124E+00
       6      0.22548000E+00      0.22039741E+00

5b28b909af10727d0e6a908a
{'atmpro': 'mls', 'band': '3b', 'commitnumber': 'a06b618', 'molecule': 'co2'}
       g                  dg                 dgs
       1      0.12800000E-02      0.12842906E-02
       2      0.42500000E-02      0.42677704E-02
       3      0.19960000E-01      0.20046012E-01
       4      0.65950000E-01      0.66214986E-01
       5      0.24303000E+00      0.24386821E+00
       6      0.47273000E+00      0.47310358E+00
       7      0.19280000E+00      0.19121514E+00

